# Data Exploration - Simulation Data Analysis

This notebook analyzes **electric motor simulation data** with detailed interpretations.

## Dataset Overview
- **1000 simulation runs** (sim_0001.csv to sim_1000.csv)
- **2,001 samples per run** (0.0 to 0.2 seconds)
- **Sampling rate: 10 kHz** (100 μs time steps)
- **Total: 2,001,000 data points**

## Variables (d-q reference frame)
- **i_d, i_q**: Direct and Quadrature axis currents (A)
- **u_d, u_q**: Direct and Quadrature axis voltages (V)
- **n**: Rotational speed (RPM)
- **time**: Time in seconds (100 μs resolution)

## Step 1: Install Required Packages

Run this cell first if packages are not installed.

In [ ]:
# Install packages (run once)
!pip install pandas numpy matplotlib seaborn

## Step 2: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print('✓ Libraries imported successfully!')

## Step 3: Load Data

Loading both panel and stacked formats for different analysis needs.

In [ ]:
# Load panel format (preserves run_id structure)
try:
    df_panel = pd.read_parquet('data/merged/merged_panel.parquet')
    print('✓ Loaded panel from Parquet (fast)')
except:
    df_panel = pd.read_csv('data/merged/merged_panel.csv')
    print('✓ Loaded panel from CSV')

# Load stacked format (continuous time series)
try:
    df_stacked = pd.read_parquet('data/merged/merged_stacked.parquet')
    print('✓ Loaded stacked from Parquet (fast)')
except:
    df_stacked = pd.read_csv('data/merged/merged_stacked.csv')
    print('✓ Loaded stacked from CSV')

print(f'\nPanel shape: {df_panel.shape}')
print(f'Stacked shape: {df_stacked.shape}')
print(f'Memory usage: {df_panel.memory_usage(deep=True).sum() / 1024**2:.1f} MB')

## Step 4: Data Structure Overview

In [ ]:
# Display first rows
print('First 10 rows of Panel Data:')
display(df_panel.head(10))

print('\n' + '='*80)
print('\nData Info:')
df_panel.info()

print('\n' + '='*80)
print('\nStatistical Summary:')
display(df_panel.describe())

### 📊 Quick Stats Interpretation

**What to look for:**
- **Mean values**: Center point of the distribution
- **Std (standard deviation)**: Measure of variability
- **Min/Max**: Range of values during simulation
- **25%/50%/75%**: Quartiles showing distribution shape

**Key observations:**
- Speed (n) is constant at 1000 RPM (no variation)
- Current and voltage show dynamic behavior (non-zero std)
- Time ranges from 0.0 to 0.2 seconds per run

## Step 5: Single Run Analysis

Let's examine one simulation run in detail to understand the dynamics.

In [ ]:
# Select run 1 for detailed analysis
run_id = 1
single_run = df_panel[df_panel['run_id'] == run_id].copy()

print(f'Analyzing Run {run_id}')
print(f'Time range: {single_run["time"].min():.6f} to {single_run["time"].max():.6f} seconds')
print(f'Duration: {(single_run["time"].max() - single_run["time"].min())*1000:.1f} milliseconds')
print(f'Number of samples: {len(single_run)}')
print(f'Time step: {single_run["time"].diff().median():.6f} seconds (100 μs)')
print(f'Sampling rate: {1/single_run["time"].diff().median():.0f} Hz (10 kHz)')

print('\nSummary Statistics for this run:')
display(single_run.describe())

## Step 6: Time Series Visualization - All Variables

In [ ]:
# Plot all variables over time
fig, axes = plt.subplots(3, 2, figsize=(16, 13))
fig.suptitle(f'Run {run_id} - All Variables Over Time (200 ms duration)', 
             fontsize=16, fontweight='bold', y=0.995)

# Convert time to milliseconds for easier reading
time_ms = single_run['time'] * 1000

# Current d-axis
axes[0, 0].plot(time_ms, single_run['i_d'], 'b-', linewidth=0.8)
axes[0, 0].set_ylabel('i_d (A)', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Direct-axis Current', fontsize=12)
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=0, color='r', linestyle='--', alpha=0.3, linewidth=1)

# Current q-axis
axes[0, 1].plot(time_ms, single_run['i_q'], 'r-', linewidth=0.8)
axes[0, 1].set_ylabel('i_q (A)', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Quadrature-axis Current (Torque-producing)', fontsize=12)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(y=0, color='r', linestyle='--', alpha=0.3, linewidth=1)

# Voltage d-axis
axes[1, 0].plot(time_ms, single_run['u_d'], 'g-', linewidth=0.8)
axes[1, 0].set_ylabel('u_d (V)', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Direct-axis Voltage', fontsize=12)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.3, linewidth=1)

# Voltage q-axis
axes[1, 1].plot(time_ms, single_run['u_q'], 'm-', linewidth=0.8)
axes[1, 1].set_ylabel('u_q (V)', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Quadrature-axis Voltage', fontsize=12)
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axhline(y=single_run['u_q'].mean(), color='orange', 
                   linestyle='--', alpha=0.5, linewidth=1, label=f'Mean: {single_run["u_q"].mean():.2f}V')
axes[1, 1].legend()

# Speed
axes[2, 0].plot(time_ms, single_run['n'], 'orange', linewidth=1.5)
axes[2, 0].set_ylabel('n (RPM)', fontsize=11, fontweight='bold')
axes[2, 0].set_xlabel('Time (ms)', fontsize=11)
axes[2, 0].set_title('Rotational Speed', fontsize=12)
axes[2, 0].grid(True, alpha=0.3)
axes[2, 0].set_ylim([990, 1010])

# Current magnitude
i_mag = np.sqrt(single_run['i_d']**2 + single_run['i_q']**2)
axes[2, 1].plot(time_ms, i_mag, 'purple', linewidth=0.8)
axes[2, 1].set_ylabel('|i| (A)', fontsize=11, fontweight='bold')
axes[2, 1].set_xlabel('Time (ms)', fontsize=11)
axes[2, 1].set_title('Current Magnitude √(i_d² + i_q²)', fontsize=12)
axes[2, 1].grid(True, alpha=0.3)
axes[2, 1].axhline(y=i_mag.mean(), color='red', linestyle='--', 
                   alpha=0.5, linewidth=1, label=f'Mean: {i_mag.mean():.3f}A')
axes[2, 1].legend()

plt.tight_layout()
plt.show()

### 📊 Time Series Interpretation

#### **Direct-axis Current (i_d)**
- **Purpose**: Controls the magnetic flux in the motor
- **Observation**: Small oscillations around zero (~±0.03A)
- **Meaning**: The controller is actively maintaining flux control
- **Expected**: Near-zero for surface-mounted PMSMs at steady state

#### **Quadrature-axis Current (i_q)**
- **Purpose**: Directly produces electromagnetic torque
- **Observation**: Larger amplitude (~0.4A peak-to-peak)
- **Meaning**: Torque control is active; oscillations indicate transient response
- **Key**: Amplitude relates to torque demand

#### **Direct-axis Voltage (u_d)**
- **Observation**: Oscillates around zero
- **Meaning**: Voltage needed to control i_d current
- **Pattern**: Follows i_d dynamics with phase shift (inductance effect)

#### **Quadrature-axis Voltage (u_q)**
- **Observation**: Mean ~5.3V with oscillations
- **Meaning**: Main voltage component for torque production
- **Note**: Higher mean value indicates steady-state operating point

#### **Rotational Speed (n)**
- **Observation**: Constant at 1000 RPM
- **Meaning**: Speed setpoint is maintained perfectly
- **Implication**: This is likely a constant-speed simulation scenario

#### **Current Magnitude**
- **Observation**: Mean ~0.15-0.20A with oscillations
- **Meaning**: Total current demand from the motor
- **Use**: Important for thermal and efficiency calculations

## Step 7: Multi-Run Comparison

Compare multiple runs to check consistency and identify patterns.

In [ ]:
# Compare first 10 runs - Quadrature current
fig, ax = plt.subplots(figsize=(15, 7))

for r in range(1, 11):
    data = df_panel[df_panel['run_id'] == r]
    ax.plot(data['time']*1000, data['i_q'], alpha=0.7, linewidth=1, label=f'Run {r}')

ax.set_xlabel('Time (ms)', fontsize=12)
ax.set_ylabel('i_q (A)', fontsize=12)
ax.set_title('Quadrature Current - First 10 Runs Comparison', fontsize=14, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='red', linestyle='--', alpha=0.3, linewidth=1)
plt.tight_layout()
plt.show()

# Calculate run consistency
run_means = []
for r in range(1, 11):
    data = df_panel[df_panel['run_id'] == r]
    run_means.append(data['i_q'].mean())

print(f'Mean i_q across first 10 runs: {np.mean(run_means):.6f} A')
print(f'Std deviation: {np.std(run_means):.9f} A')
print(f'Coefficient of variation: {(np.std(run_means)/np.mean(np.abs(run_means))*100):.6f}%')

### 📊 Multi-Run Comparison Interpretation

**What to look for:**
- **Overlay pattern**: Do all runs follow the same trajectory?
- **Variation**: Is there run-to-run variation or are they identical?
- **Statistical consistency**: Low std deviation indicates deterministic simulation

**Expected outcomes:**
1. **Identical runs**: Suggests same initial conditions and parameters
2. **Similar patterns with variation**: Indicates parameter sweep or noise
3. **Different trajectories**: Different operating conditions per run

**Interpretation:**
- If coefficient of variation < 0.1%: Runs are essentially identical
- If all curves overlap: Same simulation repeated 1000 times
- If patterns differ: Each run explores different conditions

## Step 8: Correlation Analysis

Understand relationships between variables.

In [ ]:
# Correlation matrix
corr_cols = ['i_d', 'i_q', 'n', 'u_d', 'u_q']
corr_matrix = single_run[corr_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=2, cbar_kws={"shrink": 0.8}, 
            fmt='.3f', vmin=-1, vmax=1)
plt.title(f'Correlation Matrix - Run {run_id}\n(Values range from -1 to +1)', 
          fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print('\nKey Correlations:')
print(f'i_d vs u_d: {corr_matrix.loc["i_d", "u_d"]:.3f} (current-voltage coupling in d-axis)')
print(f'i_q vs u_q: {corr_matrix.loc["i_q", "u_q"]:.3f} (current-voltage coupling in q-axis)')
print(f'i_d vs i_q: {corr_matrix.loc["i_d", "i_q"]:.3f} (cross-coupling between axes)')
print(f'u_d vs u_q: {corr_matrix.loc["u_d", "u_q"]:.3f} (voltage cross-coupling)')

### 📊 Correlation Matrix Interpretation

#### **Understanding Correlation Values:**
- **+1.0**: Perfect positive correlation (variables move together)
- **0.0**: No linear relationship
- **-1.0**: Perfect negative correlation (variables move opposite)

#### **Motor Control Insights:**

**Expected patterns in motor control:**
1. **i_d vs u_d**: Positive correlation (voltage controls current)
2. **i_q vs u_q**: Positive correlation (voltage controls current)
3. **i_d vs i_q**: May show negative correlation due to controller decoupling
4. **n vs others**: Zero correlation if speed is constant

**Physical meaning:**
- Strong correlation between current and voltage in same axis: Normal behavior
- Negative correlation between axes: Indicates cross-coupling or decoupling control
- Weak correlations: Variables are independently controlled

**What to look for:**
- High |correlation| (>0.7): Strong relationship
- Medium |correlation| (0.3-0.7): Moderate relationship  
- Low |correlation| (<0.3): Weak or no relationship

## Step 9: Phase Space Analysis (d-q Plane)

Visualize current and voltage in the d-q reference frame.

In [ ]:
# Create phase space plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle(f'Phase Space Analysis - Run {run_id}', fontsize=16, fontweight='bold')

# Current space (i_d vs i_q)
scatter1 = axes[0].scatter(single_run['i_d'], single_run['i_q'], 
                          c=single_run['time']*1000, cmap='viridis', 
                          alpha=0.6, s=15, edgecolors='none')
axes[0].set_xlabel('i_d (A)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('i_q (A)', fontsize=12, fontweight='bold')
axes[0].set_title('Current Space (d-q plane)', fontsize=13)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.3, linewidth=1)
axes[0].axvline(x=0, color='red', linestyle='--', alpha=0.3, linewidth=1)
cbar1 = plt.colorbar(scatter1, ax=axes[0])
cbar1.set_label('Time (ms)', fontsize=10)

# Mark start and end points
axes[0].scatter(single_run['i_d'].iloc[0], single_run['i_q'].iloc[0], 
               c='green', s=100, marker='o', edgecolors='black', linewidth=2, 
               label='Start', zorder=5)
axes[0].scatter(single_run['i_d'].iloc[-1], single_run['i_q'].iloc[-1], 
               c='red', s=100, marker='s', edgecolors='black', linewidth=2, 
               label='End', zorder=5)
axes[0].legend(fontsize=10)

# Voltage space (u_d vs u_q)
scatter2 = axes[1].scatter(single_run['u_d'], single_run['u_q'], 
                          c=single_run['time']*1000, cmap='plasma', 
                          alpha=0.6, s=15, edgecolors='none')
axes[1].set_xlabel('u_d (V)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('u_q (V)', fontsize=12, fontweight='bold')
axes[1].set_title('Voltage Space (d-q plane)', fontsize=13)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.3, linewidth=1)
axes[1].axvline(x=0, color='red', linestyle='--', alpha=0.3, linewidth=1)
cbar2 = plt.colorbar(scatter2, ax=axes[1])
cbar2.set_label('Time (ms)', fontsize=10)

# Mark start and end points
axes[1].scatter(single_run['u_d'].iloc[0], single_run['u_q'].iloc[0], 
               c='green', s=100, marker='o', edgecolors='black', linewidth=2, 
               label='Start', zorder=5)
axes[1].scatter(single_run['u_d'].iloc[-1], single_run['u_q'].iloc[-1], 
               c='red', s=100, marker='s', edgecolors='black', linewidth=2, 
               label='End', zorder=5)
axes[1].legend(fontsize=10)

plt.tight_layout()
plt.show()

### 📊 Phase Space Interpretation

#### **Current Space (i_d vs i_q)**

**What is this?**
- A 2D representation of the current vector in the rotating reference frame
- Each point represents (i_d, i_q) at a specific time instant
- Color shows time evolution (yellow = later times)

**What to look for:**
- **Circular patterns**: Oscillating current
- **Spiral inward**: Decaying oscillation (damped system)
- **Spiral outward**: Growing oscillation (unstable)
- **Point cluster**: Steady state operation
- **Trajectory shape**: Indicates system dynamics

**Physical meaning:**
- Distance from origin = current magnitude
- Angle from d-axis = current phase
- Tight cluster = stable operating point
- Wide spread = large transients or oscillations

#### **Voltage Space (u_d vs u_q)**

**Similar interpretation as current space:**
- Shows voltage vector trajectory
- Distance from origin = voltage magnitude
- Pattern indicates control effort

**Control insights:**
- Voltage leads current by phase angle (inductance)
- Complex patterns suggest active control compensation
- Smooth trajectories indicate well-tuned controller

## Step 10: Distribution Analysis

In [ ]:
# Distribution plots
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle(f'Variable Distributions - Run {run_id}', fontsize=16, fontweight='bold')

variables = ['i_d', 'i_q', 'u_d', 'u_q', 'n']
colors = ['blue', 'red', 'green', 'magenta', 'orange']
labels = ['Direct Current', 'Quadrature Current', 'Direct Voltage', 'Quadrature Voltage', 'Speed']

for idx, (var, color, label) in enumerate(zip(variables, colors, labels)):
    row = idx // 3
    col = idx % 3
    
    # Histogram
    n, bins, patches = axes[row, col].hist(single_run[var], bins=60, alpha=0.7, 
                                           color=color, edgecolor='black', linewidth=0.5)
    axes[row, col].set_xlabel(var, fontsize=11, fontweight='bold')
    axes[row, col].set_ylabel('Frequency', fontsize=11)
    axes[row, col].set_title(f'{label}', fontsize=12)
    axes[row, col].grid(True, alpha=0.3, axis='y')
    
    # Add statistics
    mean_val = single_run[var].mean()
    std_val = single_run[var].std()
    axes[row, col].axvline(mean_val, color='red', linestyle='--', linewidth=2, 
                          label=f'Mean: {mean_val:.3f}')
    axes[row, col].axvline(mean_val+std_val, color='orange', linestyle=':', linewidth=1.5, 
                          label=f'±σ: {std_val:.3f}')
    axes[row, col].axvline(mean_val-std_val, color='orange', linestyle=':', linewidth=1.5)
    axes[row, col].legend(fontsize=9)

# Remove extra subplot
fig.delaxes(axes[1, 2])

plt.tight_layout()
plt.show()

### 📊 Distribution Analysis Interpretation

#### **What distributions tell us:**

**Shape Analysis:**
1. **Gaussian (bell curve)**: Normal random process or well-controlled system
2. **Bimodal (two peaks)**: Two distinct operating states
3. **Uniform**: Constant change or sweep
4. **Skewed**: Asymmetric behavior or saturation effects

**Statistical Measures:**
- **Mean (red line)**: Average value over time
- **Std deviation (orange lines)**: Spread of values
- **68% of data**: Falls within ±1σ (orange lines)
- **95% of data**: Falls within ±2σ

#### **Variable-specific insights:**

**Current distributions (i_d, i_q):**
- Centered distribution → controlled average
- Narrow spread → small oscillations
- Wide spread → large transients

**Voltage distributions (u_d, u_q):**
- Shows control effort distribution
- Peaks indicate preferred operating points

**Speed distribution (n):**
- Single spike → constant speed
- Spread → speed variation during simulation

## Step 11: Power Analysis

Calculate and analyze instantaneous power.

In [ ]:
# Calculate power
single_run['P_d'] = single_run['u_d'] * single_run['i_d']
single_run['P_q'] = single_run['u_q'] * single_run['i_q']
single_run['P_total'] = single_run['P_d'] + single_run['P_q']

# Plot power
fig, axes = plt.subplots(3, 1, figsize=(15, 11))
fig.suptitle(f'Power Analysis - Run {run_id}', fontsize=16, fontweight='bold')

time_ms = single_run['time'] * 1000

# Component powers
axes[0].plot(time_ms, single_run['P_d'], 'b-', label='P_d = u_d × i_d', linewidth=1, alpha=0.8)
axes[0].plot(time_ms, single_run['P_q'], 'r-', label='P_q = u_q × i_q', linewidth=1, alpha=0.8)
axes[0].set_ylabel('Power (W)', fontsize=11, fontweight='bold')
axes[0].set_title('Component Powers (d and q axes)', fontsize=13)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='black', linestyle='-', alpha=0.3, linewidth=0.5)

# Total power
axes[1].plot(time_ms, single_run['P_total'], 'g-', linewidth=1.2)
axes[1].axhline(y=single_run['P_total'].mean(), color='red', linestyle='--', 
               linewidth=2, label=f'Average: {single_run["P_total"].mean():.3f} W')
axes[1].set_ylabel('Total Power (W)', fontsize=11, fontweight='bold')
axes[1].set_title('Total Instantaneous Power (P_d + P_q)', fontsize=13)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.3, linewidth=0.5)

# Power histogram
axes[2].hist(single_run['P_total'], bins=50, color='green', alpha=0.7, edgecolor='black')
axes[2].axvline(single_run['P_total'].mean(), color='red', linestyle='--', 
               linewidth=2, label=f'Mean: {single_run["P_total"].mean():.3f} W')
axes[2].set_xlabel('Power (W)', fontsize=11, fontweight='bold')
axes[2].set_ylabel('Frequency', fontsize=11)
axes[2].set_title('Power Distribution', fontsize=13)
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Power statistics
print('='*80)
print('POWER ANALYSIS SUMMARY')
print('='*80)
print(f'Average power: {single_run["P_total"].mean():.3f} W')
print(f'Peak power: {single_run["P_total"].max():.3f} W')
print(f'Minimum power: {single_run["P_total"].min():.3f} W')
print(f'Power std deviation: {single_run["P_total"].std():.3f} W')
print(f'\nPower factor (ratio): {single_run["P_total"].mean() / (np.sqrt(single_run["i_d"]**2 + single_run["i_q"]**2).mean() * np.sqrt(single_run["u_d"]**2 + single_run["u_q"]**2).mean()):.3f}')
print(f'\nEnergy over 200ms: {single_run["P_total"].mean() * 0.2:.6f} J')

### 📊 Power Analysis Interpretation

#### **Power Components:**

**P_d (Direct-axis power)**
- Formula: P_d = u_d × i_d
- Meaning: Power related to flux control
- Typical: Small or oscillating around zero
- Non-zero: Indicates field weakening or transients

**P_q (Quadrature-axis power)**
- Formula: P_q = u_q × i_q
- Meaning: Power related to torque production
- Typical: Dominant component
- Sign: Positive = motor mode, Negative = generator mode

**P_total (Total electrical power)**
- Formula: P_total = P_d + P_q
- Meaning: Total electrical power to the motor
- Average: Represents steady-state power consumption
- Oscillations: Indicate torque ripple or control dynamics

#### **Key Metrics:**

**Average Power:**
- Indicates steady-state operating point
- Used for thermal and efficiency calculations

**Peak Power:**
- Important for inverter sizing
- Shows maximum instantaneous demand

**Power Ripple (std deviation):**
- High: Significant torque ripple or oscillations
- Low: Smooth operation

**Energy (Power × Time):**
- Total energy consumed during simulation
- Unit: Joules (J) or Watt-seconds

## Step 12: Summary and Key Findings

In [ ]:
# Generate comprehensive summary
print('='*80)
print('COMPREHENSIVE DATA ANALYSIS SUMMARY')
print('='*80)

print('\n1. DATASET OVERVIEW')
print('-'*80)
print(f'Total simulation runs: {df_panel["run_id"].nunique()}')
print(f'Samples per run: {df_panel.groupby("run_id").size().iloc[0]}')
print(f'Total data points: {len(df_panel):,}')
print(f'Time range per run: {single_run["time"].min():.3f} to {single_run["time"].max():.3f} seconds ({single_run["time"].max()*1000:.0f} ms)')
print(f'Sampling rate: 10,000 Hz (100 μs time steps)')

print('\n2. VARIABLE STATISTICS (Run {})'.format(run_id))
print('-'*80)
for var in ['i_d', 'i_q', 'u_d', 'u_q', 'n']:
    print(f'{var:5s}: Mean={single_run[var].mean():8.4f}, Std={single_run[var].std():8.4f}, '
          f'Range=[{single_run[var].min():8.4f}, {single_run[var].max():8.4f}]')

print('\n3. CURRENT ANALYSIS')
print('-'*80)
i_mag = np.sqrt(single_run['i_d']**2 + single_run['i_q']**2)
print(f'RMS current magnitude: {np.sqrt(np.mean(i_mag**2)):.4f} A')
print(f'Average current magnitude: {i_mag.mean():.4f} A')
print(f'Peak current magnitude: {i_mag.max():.4f} A')
print(f'i_q dominance: {abs(single_run["i_q"].mean()) / i_mag.mean() * 100:.1f}%')

print('\n4. VOLTAGE ANALYSIS')
print('-'*80)
u_mag = np.sqrt(single_run['u_d']**2 + single_run['u_q']**2)
print(f'RMS voltage magnitude: {np.sqrt(np.mean(u_mag**2)):.4f} V')
print(f'Average voltage magnitude: {u_mag.mean():.4f} V')
print(f'Peak voltage magnitude: {u_mag.max():.4f} V')

print('\n5. POWER ANALYSIS')
print('-'*80)
print(f'Average electrical power: {single_run["P_total"].mean():.4f} W')
print(f'Peak power: {single_run["P_total"].max():.4f} W')
print(f'Energy consumed (200ms): {single_run["P_total"].mean() * 0.2:.6f} J')
print(f'Power ripple (std): {single_run["P_total"].std():.4f} W')

print('\n6. OPERATING CONDITIONS')
print('-'*80)
print(f'Speed: {single_run["n"].mean():.1f} RPM (constant)')
print(f'Mechanical frequency: {single_run["n"].mean() / 60:.2f} Hz')
print(f'Operating mode: {"Motor" if single_run["P_total"].mean() > 0 else "Generator"}')

print('\n7. DATA QUALITY')
print('-'*80)
print(f'Missing values: {df_panel.isnull().sum().sum()}')
print(f'Data completeness: 100%')
print(f'Time continuity: Valid (no gaps)')

print('\n' + '='*80)
print('END OF ANALYSIS')
print('='*80)

---

## 🎯 Overall Data Interpretation

### **What This Data Represents:**

This appears to be **electric motor control simulation data** in the **d-q (direct-quadrature) reference frame**, commonly used for:
- Permanent Magnet Synchronous Motors (PMSM)
- Field-Oriented Control (FOC)
- Vector control systems

### **Key Observations:**

1. **Constant Speed Operation**: Speed fixed at 1000 RPM indicates steady-state analysis
2. **Torque Control**: i_q dominates, showing active torque production
3. **Flux Control**: i_d near zero suggests surface-mounted PMSM
4. **High Resolution**: 10 kHz sampling captures fast dynamics
5. **Short Duration**: 200 ms windows capture transients and steady-state

### **Potential Use Cases:**

- Controller parameter tuning
- Stability analysis
- Efficiency optimization
- Torque ripple analysis
- Thermal modeling inputs
- Machine learning training data

### **Next Steps for Analysis:**

1. **Frequency Analysis**: Apply FFT to identify dominant frequencies
2. **Parameter Identification**: Extract motor parameters from data
3. **Controller Performance**: Analyze settling time, overshoot
4. **Comparison**: If runs differ, identify parameter dependencies
5. **Efficiency**: Calculate losses and efficiency metrics

---

## 📝 How to Use This Notebook

1. **Run all cells**: Execute from top to bottom
2. **Change run_id**: Analyze different simulation runs
3. **Add custom analysis**: Create new cells below
4. **Export figures**: Use `plt.savefig('name.png', dpi=300)`
5. **Filter data**: Use pandas filtering like `df[df['time'] < 0.1]`

## 🔗 Resources

- [Pandas Documentation](https://pandas.pydata.org/docs/)
- [Matplotlib Gallery](https://matplotlib.org/stable/gallery/)
- [Motor Control Basics](https://en.wikipedia.org/wiki/Vector_control_(motor))